In [4]:
from bs4 import BeautifulSoup     
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import pandas as pd
import xlwt
import math

In [5]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
    time.sleep(1)

In [6]:
query_txt=input('유튜브에서 검색할 주제 키워드를 입력하세요: ')
videonum=input('위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?: ')
count=int(input('각 동영상에서 추출할 댓글은 몇건 입니까?: '))
f_name = input('파일로 저장할 폴더 이름을 쓰세요(예:c:\data\): ')

# query_txt='yoasobi'
# videonum=3
# count=30
# f_name = '/Users/hwangseho/Desktop/SNSbigdata_sourcecode/final_res/Q5/'

pagedown=math.floor(count/20)

In [7]:
path = '/opt/homebrew/Caskroom/chromedriver/112.0.5615.49/chromedriver'
driver = webdriver.Chrome(path)
driver.get("https://youtube.com/")
time.sleep(1)

/var/folders/g7/dh8v6rd91kz0fm7zl5kcjvq40000gn/T/ipykernel_41447/2778302865.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [8]:
element=driver.find_element(By.NAME, 'search_query').click()
driver.find_element(By.NAME, 'search_query').send_keys(query_txt)
time.sleep(1)

element=driver.find_element(By.XPATH, '//*[@id="search-icon-legacy"]').click()
time.sleep(1)

In [9]:
element=driver.find_element(By.XPATH, '//*[@id="container"]/ytd-toggle-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
time.sleep(1)
element=driver.find_element(By.XPATH, '//*[@id="label"]/yt-formatted-string').click()
time.sleep(1)
element=driver.find_element(By.XPATH, '//*[@id="container"]/ytd-toggle-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
time.sleep(1)
element=driver.find_element(By.XPATH, '//*[@id="dismiss-x"]').click()
time.sleep(1)

In [10]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
content_total = soup.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')

In [11]:
videoLinks=[]

for i in content_total:
    videoLinks.append("https://www.youtube.com"+i['href'])


In [12]:
def makelists(content_list,link):
    global count
    global num,numList,vidLinks,nickNames,wDates,contents
    for i in range(count):
        numList.append(num)
        
        vidLinks.append(link)

        nickName=content_list[i].find('span','style-scope ytd-comment-renderer style-scope ytd-comment-renderer')
        if nickName==None:
            nickName=content_list[i].find('span','channel-owner style-scope ytd-comment-renderer style-scope ytd-comment-renderer')
        nickNames.append(nickName.get_text( ))
        
        writeDate=content_list[i].find('a','yt-simple-endpoint style-scope yt-formatted-string')
        wDates.append(writeDate.get_text( ))

        content=content_list[i].find('yt-formatted-string','style-scope ytd-comment-renderer')
        contents.append(content.get_text( ))

        num+=1

In [ ]:
num=1
numList=[]
vidLinks=[]
nickNames=[]
wDates=[]
contents=[]

for i in range(int(videonum)):
    driver.get(videoLinks[i])
    time.sleep(3)
    for i in range(pagedown):
        scroll_down(driver)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find_all('ytd-comment-thread-renderer','style-scope ytd-item-section-renderer')
    
    makelists(content_list,videoLinks[i])

driver.quit()

In [14]:
dataFrame=pd.DataFrame()
dataFrame['']=numList
dataFrame['URL 주소']=vidLinks
dataFrame['댓글작성자명']=nickNames
dataFrame['댓글작성일자']=wDates
dataFrame['댓글 내용']=contents

dataFrame.to_csv(f_name+'Q5.csv',encoding="utf-8-sig",index=False)
dataFrame.to_excel(f_name+'Q5.xls',index=False)

f=open(f_name+'Q5.txt','a',encoding='UTF-8')
f.write(str(numList))
f.write(str(vidLinks))
f.write(str(nickNames))
f.write(str(wDates))
f.write(str(contents))
f.close( )

/var/folders/g7/dh8v6rd91kz0fm7zl5kcjvq40000gn/T/ipykernel_41447/4144071268.py:9: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  dataFrame.to_excel(f_name+'Q5.xls',index=False)
